In [1]:
import pandas as pd
from ipyfilechooser import FileChooser
from IPython.display import display
from pathlib import Path

# --- File chooser (run this cell first) ---
file_chooser = FileChooser()
file_chooser.title = '<b>Select your Excel file</b>'
file_chooser.filter_pattern = '*.xlsx'
display(file_chooser)

print("Select a file, then run the next cell.")


FileChooser(path='/Users/liyue/Documents/PhD@Purdue/my_projects/RCE_project_EF_YL/data_updating!', filename=''…

Select a file, then run the next cell.


In [2]:
# --- Run this cell AFTER selecting a file ---
file_path = file_chooser.selected
if not file_path:
    raise ValueError("No file selected. Please select a file above and re-run this cell.")

input_path = Path(file_path)
df = pd.read_excel(input_path)

# --- Helper: Excel letter ("AC", "BO", etc.) -> zero-based column index ---
def col_letter_to_index(col_letter: str) -> int:
    col_letter = col_letter.strip().upper()
    n = 0
    for ch in col_letter:
        n = n * 26 + (ord(ch) - ord('A') + 1)
    return n - 1  # zero-based

# --- Helper: select columns by Excel letters, preserving order ---
def select_cols_by_letters(df_in: pd.DataFrame, letters) -> pd.DataFrame:
    idxs = [col_letter_to_index(l) for l in letters]
    max_ok = df_in.shape[1] - 1
    for ltr, idx in zip(letters, idxs):
        if idx > max_ok:
            raise IndexError(f"Column {ltr} (index {idx}) is out of range for this file.")
    return df_in.iloc[:, idxs]

# Step 1: keep only rows where column BO contains "exprmt" (case-insensitive, substring)
bo_idx = col_letter_to_index("BO")
df_step1 = df[df.iloc[:, bo_idx].astype(str).str.contains("exprmt", na=False, case=False)]

# Step 2: from step1, keep rows where AZ == "object_214"
az_idx = col_letter_to_index("AZ")
df_step2 = df_step1[df_step1.iloc[:, az_idx].astype(str) == "object-214"]

# Step 3: from step2, test whether AC includes "list1" or "list2" (case-insensitive, substring)
ac_idx = col_letter_to_index("AC")
ac_series = df_step2.iloc[:, ac_idx].astype(str)
mask_list1 = ac_series.str.contains("list1", na=False, case=False)
mask_list2 = ac_series.str.contains("list2", na=False, case=False)

# Step 4: output with different column sets depending on AC
cols_for_list1 = ["AC", "AL", "AZ", "BE", "BF", "BG", "BH", "BQ", "BS"]
cols_for_list2 = ["AC", "AL", "AZ", "BJ", "BK", "BL", "BM", "BR", "BT"]

out_list1 = select_cols_by_letters(df_step2[mask_list1], cols_for_list1)
out_list2 = select_cols_by_letters(df_step2[mask_list2], cols_for_list2)

# Save next to the input with an affix "_filtered"
output_path = input_path.with_name(f"{input_path.stem}_filtered.xlsx")

with pd.ExcelWriter(output_path) as writer:
    out_list1.to_excel(writer, index=False, sheet_name="list1")
    out_list2.to_excel(writer, index=False, sheet_name="list2")

print(f"Done. Wrote {len(out_list1)} rows to 'list1' and {len(out_list2)} rows to 'list2'.")
print(f"Saved to: {output_path}")


Done. Wrote 32 rows to 'list1' and 0 rows to 'list2'.
Saved to: /Users/liyue/Documents/PhD@Purdue/my_projects/RCE_project_EF_YL/data_updating!/data_exp_187289-v2_13860231_pilot01/data_exp_187289-v2_task-hdd6-13860231_filtered.xlsx
